# DCF Modeling

In [1]:
import excelify as el

# Step 0: Assumptions

In [2]:
previous_tax_rates = [6858 / 20564, 4915 / 20116, 4281 / 11460]
assumptions_df = el.ExcelFrame(
    {
        "Company Name": ["Walmart Inc."],
        "Ticker": ["WMT"],
        "Current Share Price": [139.43],
        "Effective Tax Rate": [sum(previous_tax_rates) / len(previous_tax_rates)],
        "Last Fiscal Year": ["2021-01-31"],
    },
)

# TODO: Make this more ergonomic.
assumptions_df["Company Name"].set_attributes({"bgcolor": "blue"})
assumptions_df["Current Share Price"].set_attributes({"bgcolor": "#000"})
assumptions_df.transpose(
    include_header=True, header_name="Name", column_names=["Constants"]
)

Name,Constants
Company Name,Walmart Inc.
Ticker,WMT
Current Share Price,139.43
Effective Tax Rate,0.32
Last Fiscal Year,2021-01-31


## Step 1: Unlevered Free Cash Flow

In [3]:
years = ["2019", "2020", "2021"]

In [4]:
def nopat(operating_income, taxes):
    return operating_income + taxes


def deferred_taxes(taxes, percent_book_taxes):
    return -taxes * percent_book_taxes


def unlevered_free_cash_flow(
    nopat, d_and_a, deferred_taxes, other_operating, change_in_working_capial, capex
):
    return (
        nopat
        + d_and_a
        + deferred_taxes
        + other_operating
        + change_in_working_capial
        + capex
    )

def ebitda(ebit, da):
    return ebit + da

def growth_rate(col: el.Col):
    return ((col / col.prev(1)) - 1)

In [5]:
sqft_columns = [
    "Retail Square Feet",
    "Sales per Square Foot",
    "COGS and OpEx per Square Foot",
    "Maintenance CapEx per Square Foot",
    "D&A per Square Foot",
    "Growth CapEx per New Square Foot",
]

sqft_with_growth_rate_columns = [
    [col, f"{col} Growth Rate"]
    for col in sqft_columns
]
sqft_with_growth_rate_columns = [
    col
    for cols in sqft_with_growth_rate_columns
    for col in cols
] + ["Membership & Other Income Growth Rate"]

revenue_columns = [
    "Net Sales",
    "Membership & Other Income",
    "Total Revenue",
    "Revenue Growth",
]
cash_flow_columns = [
    "Operating Income (EBIT)",
    "Operating (EBIT) Margin",
    "Taxes, Excluding Effect of Interest",
    "Net Operating Profit After Tax (NOPAT)",
    "Adjustments for Non-Cash Charges",
    "Depreciation & Amortization",
    "Depreciation & Amortization (% Revenue)",
    "Deferred Taxes",
    "Deferred Taxes (% Book Taxes)",
    "Other Operating Activities",
    "Other Operating Activities (% Revenue)",
    "Change in Working Capital",
    "Change in Working Capital (% Change in Revenue)",
    "Capital Expenditures",
    "Capital Expenditures (% Revenue)",
    "Annual Unlevered Free Cash Flow",
    "Annual Unlevered Free Cash Flow Growth Rate",
    "Period",
    "PV of Unlevered FCF",
    "EBITDA",
    "EBITDA Growth Rate",
]
columns = ["years"] + sqft_with_growth_rate_columns + revenue_columns + cash_flow_columns
columns

['years',
 'Retail Square Feet',
 'Retail Square Feet Growth Rate',
 'Sales per Square Foot',
 'Sales per Square Foot Growth Rate',
 'COGS and OpEx per Square Foot',
 'COGS and OpEx per Square Foot Growth Rate',
 'Maintenance CapEx per Square Foot',
 'Maintenance CapEx per Square Foot Growth Rate',
 'D&A per Square Foot',
 'D&A per Square Foot Growth Rate',
 'Growth CapEx per New Square Foot',
 'Growth CapEx per New Square Foot Growth Rate',
 'Membership & Other Income Growth Rate',
 'Net Sales',
 'Membership & Other Income',
 'Total Revenue',
 'Revenue Growth',
 'Operating Income (EBIT)',
 'Operating (EBIT) Margin',
 'Taxes, Excluding Effect of Interest',
 'Net Operating Profit After Tax (NOPAT)',
 'Adjustments for Non-Cash Charges',
 'Depreciation & Amortization',
 'Depreciation & Amortization (% Revenue)',
 'Deferred Taxes',
 'Deferred Taxes (% Book Taxes)',
 'Other Operating Activities',
 'Other Operating Activities (% Revenue)',
 'Change in Working Capital',
 'Change in Working Ca

In [6]:
historical_ufcf_df = el.ExcelFrame.empty(columns=columns, height=3)
ca = el.ColumnAutocompleter(columns)

In [7]:
def growth_rate(col: str, *, col_name = None, prev_df = None):
    if col_name is None:
        col_name = f"{col} Growth Rate"

    def fn(idx):
        if idx == 0 and prev_df is not None:
            return (el.col(col) / el.SingleCellExpr(prev_df[col][-1]) - 1)
        else:
            return (el.col(col) / el.col(col).prev(1) - 1).alias(col_name)
    return el.Map(fn).alias(col_name)

def maintenance_capex_per_sqft_formula(idx):
    if idx == 0:
        return -el.col(ca.Capital_Expenditures) / 1158
    else:
        return -el.col(ca.Capital_Expenditures) / el.col(ca.Retail_Square_Feet).prev(1)

def percent_revenue(col: str):
    return (el.col(col) / el.col(ca.Total_Revenue)).alias(f"{col} (% Revenue)")

In [8]:
historical_ufcf_df = historical_ufcf_df.with_columns(
    el.lit(years).alias("years"),
    el.lit([1_129, 1_129, 1_121]).alias("Retail Square Feet"),
    el.lit([510_329.0, 519_926.0, 555_233.0]).alias("Net Sales"),
    el.lit([4076.0, 4038.0, 3918.0]).alias("Membership & Other Income"),
    el.lit([21_957.0, 20_568.0, 22_548.0]).alias("Operating Income (EBIT)"),
    el.lit([10_678.0, 10_987.0, 11_152.0]).alias("Depreciation & Amortization"),
    el.lit([-499 / 4281, 320 / 4915, 1911 / 6858]).alias("% Book Taxes"),
    el.lit([1_734, 1_981, 1_521]).alias("Other Operating Activities"),
    el.lit([295, -327, 7972]).alias("Change in Working Capital"),
    el.lit([-10_344, -10_705, -10_264]).alias("Capital Expenditures"),
    el.lit([-499 / 4281, 320 / 4915, 1911 / 6858]).alias(
        "Deferred Taxes (% Book Taxes)"
    ),
)

historical_ufcf_df = (
    historical_ufcf_df.with_row_index()
    .with_columns(
        growth_rate(ca.Retail_Square_Feet),
        (el.col(ca.Net_Sales) / el.col(ca.Retail_Square_Feet)).alias(
            ca.Sales_per_Square_Foot
        ),
        growth_rate(ca.Sales_per_Square_Foot),
        (el.col(ca.Net_Sales) - el.col(ca.Operating_Income__EBIT_)),
        (
            (el.col(ca.Net_Sales) - el.col(ca.Operating_Income__EBIT_))
            / el.col(ca.Retail_Square_Feet)
        ).alias(ca.COGS_and_OpEx_per_Square_Foot),
        growth_rate(ca.COGS_and_OpEx_per_Square_Foot),
        el.Map(maintenance_capex_per_sqft_formula).alias(
            ca.Maintenance_CapEx_per_Square_Foot
        ),
        growth_rate(ca.Maintenance_CapEx_per_Square_Foot),
        (
            el.col(ca.Depreciation_and_Amortization) / el.col(ca.Retail_Square_Feet)
        ).alias(ca.DandA_per_Square_Foot),
        growth_rate(ca.DandA_per_Square_Foot),
        growth_rate(ca.Membership_and_Other_Income),
        (el.col(ca.Net_Sales) + el.col(ca.Membership_and_Other_Income)).alias(
            ca.Total_Revenue
        ),
        growth_rate(ca.Total_Revenue, col_name="Revenue Growth"),
        (el.col(ca.Operating_Income__EBIT_) / el.col(ca.Total_Revenue)).alias(
            ca.Operating__EBIT__Margin
        ),
        (
            -el.col(ca.Operating_Income__EBIT_)
            # TODO: Slightly ugly.
            * el.SingleCellExpr(assumptions_df["Effective Tax Rate"][0])
        ).alias(ca.Taxes__Excluding_Effect_of_Interest),
        (
            el.col(ca.Operating_Income__EBIT_)
            + el.col(ca.Taxes__Excluding_Effect_of_Interest)
        ).alias(ca.Net_Operating_Profit_After_Tax__NOPAT_),
        percent_revenue(ca.Depreciation_and_Amortization),
        (
            -el.col(ca.Deferred_Taxes__percent_Book_Taxes_)
            * el.col(ca.Taxes__Excluding_Effect_of_Interest)
        ).alias(ca.Deferred_Taxes),
        percent_revenue(ca.Other_Operating_Activities),
        (
            el.col(ca.Change_in_Working_Capital)
            / (el.col(ca.Total_Revenue) - el.col(ca.Total_Revenue).prev(1))
        ).alias(ca.Change_in_Working_Capital__percent_Change_in_Revenue_),
        percent_revenue(ca.Capital_Expenditures),
        (
            el.col(ca.Net_Operating_Profit_After_Tax__NOPAT_)
            + el.col(ca.Depreciation_and_Amortization)
            + el.col(ca.Deferred_Taxes)
            + el.col(ca.Other_Operating_Activities)
            + el.col(ca.Change_in_Working_Capital)
            + el.col(ca.Capital_Expenditures)
        ).alias(ca.Annual_Unlevered_Free_Cash_Flow),
        growth_rate(ca.Annual_Unlevered_Free_Cash_Flow),
        (
            el.col(ca.Operating_Income__EBIT_) + el.col(ca.Depreciation_and_Amortization)
        ).alias(ca.EBITDA),
        growth_rate(ca.EBITDA),
    )
    .select(columns)
)

historical_ufcf_df.evaluate().transpose(
    include_header=True,
    header_name="Name",
    column_names=years,
)

Name,2019,2020,2021
years,2019.0,2020.0,2021.0
Retail Square Feet,1129.0,1129.0,1121.0
Retail Square Feet Growth Rate,None,0.0,-0.01
Sales per Square Foot,452.02,460.52,495.3
Sales per Square Foot Growth Rate,None,0.02,0.08
COGS and OpEx per Square Foot,432.57,442.3,475.19
COGS and OpEx per Square Foot Growth Rate,None,0.02,0.07
Maintenance CapEx per Square Foot,8.93,9.48,9.09
Maintenance CapEx per Square Foot Growth Rate,None,0.06,-0.04
D&A per Square Foot,9.46,9.73,9.95


In [9]:
def projected_value(col, growth_rate_col):
    def fn(idx):
        if idx == 0:
            return el.SingleCellExpr(historical_ufcf_df[col][-1]) * (
                1 + el.col(growth_rate_col)
            )
        else:
            return el.col(col).prev(1) * (1 + el.col(growth_rate_col))

    return el.Map(fn).alias(col)


def growth_capex_per_new_sqft_formula(idx):
    if idx == 0:
        return el.ConstantExpr(150.0)
    else:
        return el.col(ca.Growth_CapEx_per_New_Square_Foot).prev(1) * (
            1 + el.col(ca.Growth_CapEx_per_New_Square_Foot_Growth_Rate)
        )


def change_in_working_capital_formula(idx):
    if idx == 0:
        return el.col(ca.Change_in_Working_Capital__percent_Change_in_Revenue_) * (
            el.col(ca.Total_Revenue)
            - el.SingleCellExpr(historical_ufcf_df[ca.Total_Revenue][-1])
        )
    else:
        return el.col(ca.Change_in_Working_Capital__percent_Change_in_Revenue_) * (
            el.col(ca.Total_Revenue) - el.col(ca.Total_Revenue).prev(1)
        )


def capital_expenditures_formula(idx):
    prev_retail_square_feet = (
        el.col(ca.Retail_Square_Feet).prev(1)
        if idx > 0
        else el.SingleCellExpr(historical_ufcf_df[ca.Retail_Square_Feet][-1])
    )
    return -el.col(
        ca.Maintenance_CapEx_per_Square_Foot
    ) * prev_retail_square_feet - el.col(ca.Growth_CapEx_per_New_Square_Foot) * (
        el.col(ca.Retail_Square_Feet) - prev_retail_square_feet
    )

In [15]:
years = [str(2022 + idx) for idx in range(10)]
projected_ufcf_df = el.ExcelFrame.empty(columns=historical_ufcf_df.columns, height=10)

projected_ufcf_df = projected_ufcf_df.with_columns(
    el.lit(years).alias(ca.years),
    projected_value(ca.Retail_Square_Feet, ca.Retail_Square_Feet_Growth_Rate),
    el.lit([0.02, 0.02, 0.015, 0.015, 0.01, 0.01, 0.005, 0.005, 0.005, 0.005]).alias(
        ca.Retail_Square_Feet_Growth_Rate
    ),
    projected_value(ca.Sales_per_Square_Foot, ca.Sales_per_Square_Foot_Growth_Rate),
    el.lit([0.03, 0.03, 0.025, 0.025, 0.02, 0.02, 0.015, 0.015, 0.01, 0.01]).alias(
        ca.Sales_per_Square_Foot_Growth_Rate
    ),
    projected_value(
        ca.COGS_and_OpEx_per_Square_Foot, ca.COGS_and_OpEx_per_Square_Foot_Growth_Rate
    ),
    el.lit([0.03, 0.03, 0.025, 0.025, 0.02, 0.02, 0.015, 0.015, 0.01, 0.01]).alias(
        ca.COGS_and_OpEx_per_Square_Foot_Growth_Rate
    ),
    projected_value(
        ca.Maintenance_CapEx_per_Square_Foot,
        ca.Maintenance_CapEx_per_Square_Foot_Growth_Rate,
    ),
    el.lit([0.03, 0.03, 0.025, 0.025, 0.02, 0.02, 0.015, 0.015, 0.01, 0.01]).alias(
        ca.Maintenance_CapEx_per_Square_Foot_Growth_Rate
    ),
    projected_value(ca.DandA_per_Square_Foot, ca.DandA_per_Square_Foot_Growth_Rate),
    el.lit([0.025, 0.025, 0.02, 0.02, 0.015, 0.015, 0.01, 0.01, 0.008, 0.008]).alias(
        ca.DandA_per_Square_Foot_Growth_Rate
    ),
    el.Map(growth_capex_per_new_sqft_formula).alias(
        ca.Growth_CapEx_per_New_Square_Foot
    ),
    el.lit([None, 0.03, 0.03, 0.025, 0.025, 0.02, 0.02, 0.015, 0.015, 0.01]).alias(
        ca.Growth_CapEx_per_New_Square_Foot_Growth_Rate
    ),
    el.lit([0.03, 0.03, 0.025, 0.025, 0.02, 0.02, 0.015, 0.015, 0.01, 0.01]).alias(
        ca.Membership_and_Other_Income_Growth_Rate
    ),
    (el.col(ca.Retail_Square_Feet) * el.col(ca.Sales_per_Square_Foot)).alias(
        ca.Net_Sales
    ),
    projected_value(
        ca.Membership_and_Other_Income, ca.Membership_and_Other_Income_Growth_Rate
    ),
    (el.col(ca.Net_Sales) + el.col(ca.Membership_and_Other_Income)).alias(
        ca.Total_Revenue
    ),
    growth_rate(
        ca.Total_Revenue, col_name=ca.Revenue_Growth, prev_df=historical_ufcf_df
    ),
    (
        el.col(ca.Net_Sales)
        - el.col(ca.COGS_and_OpEx_per_Square_Foot) * el.col(ca.Retail_Square_Feet)
    ).alias(ca.Operating_Income__EBIT_),
    (el.col(ca.Operating_Income__EBIT_) / el.col(ca.Total_Revenue)).alias(
        ca.Operating__EBIT__Margin
    ),
    (
        -el.col(ca.Operating_Income__EBIT_)
        * el.SingleCellExpr(assumptions_df["Effective Tax Rate"][0])
    ).alias(ca.Taxes__Excluding_Effect_of_Interest),
    (
        el.col(ca.Operating_Income__EBIT_)
        + el.col(ca.Taxes__Excluding_Effect_of_Interest)
    ).alias(ca.Net_Operating_Profit_After_Tax__NOPAT_),
    (el.col(ca.Retail_Square_Feet) * el.col(ca.DandA_per_Square_Foot)).alias(
        ca.Depreciation_and_Amortization
    ),
    percent_revenue(ca.Depreciation_and_Amortization),
    (
        -el.col(ca.Deferred_Taxes__percent_Book_Taxes_)
        * el.col(ca.Taxes__Excluding_Effect_of_Interest)
    ).alias(ca.Deferred_Taxes),
    el.lit([0.075, 0.07, 0.065, 0.06, 0.055, 0.05, 0.05, 0.05, 0.05, 0.05]).alias(
        ca.Deferred_Taxes__percent_Book_Taxes_
    ),
    (
        el.col(ca.Total_Revenue)
        * el.col(ca.Other_Operating_Activities__percent_Revenue_)
    ).alias(ca.Other_Operating_Activities),
    # TODO: Improve this.
    (
        sum(
            [
                el.SingleCellExpr(
                    historical_ufcf_df[ca.Other_Operating_Activities__percent_Revenue_][
                        i
                    ]
                )
                for i in range(3)
            ]
        )
        / 3
    ).alias(ca.Other_Operating_Activities__percent_Revenue_),
    el.Map(change_in_working_capital_formula).alias(ca.Change_in_Working_Capital),
    el.lit([0.1, 0.095, 0.09, 0.085, 0.08, 0.075, 0.075, 0.075, 0.075, 0.075]).alias(
        ca.Change_in_Working_Capital__percent_Change_in_Revenue_
    ),
    el.Map(capital_expenditures_formula).alias(ca.Capital_Expenditures),
    percent_revenue(ca.Capital_Expenditures),
    (
        el.col(ca.Net_Operating_Profit_After_Tax__NOPAT_)
        + el.col(ca.Depreciation_and_Amortization)
        + el.col(ca.Deferred_Taxes)
        + el.col(ca.Other_Operating_Activities)
        + el.col(ca.Change_in_Working_Capital)
        + el.col(ca.Capital_Expenditures)
    ).alias(ca.Annual_Unlevered_Free_Cash_Flow),
    growth_rate(ca.Annual_Unlevered_Free_Cash_Flow, prev_df=historical_ufcf_df),
    el.lit([i + 1 for i in range(10)]).alias(ca.Period),
    (
        el.col(ca.Annual_Unlevered_Free_Cash_Flow)
        / ((1 + 0.04365) ** el.col(ca.Period))
    ).alias(ca.PV_of_Unlevered_FCF),
    (el.col(ca.Operating_Income__EBIT_) + el.col(ca.Depreciation_and_Amortization)).alias(
        ca.EBITDA
    ),
    growth_rate(ca.EBITDA, prev_df=historical_ufcf_df),
)

projected_ufcf_df.evaluate().transpose(
    include_header=True, header_name="Name", column_names=years
)

Name,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031
years,2022.0,2023.0,2024.0,2025.0,2026.0,2027.0,2028.0,2029.0,2030.0,2031.0
Retail Square Feet,1143.42,1166.29,1183.78,1201.54,1213.55,1225.69,1231.82,1237.98,1244.17,1250.39
Retail Square Feet Growth Rate,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
Sales per Square Foot,510.16,525.47,538.6,552.07,563.11,574.37,582.99,591.73,597.65,603.62
Sales per Square Foot Growth Rate,0.03,0.03,0.03,0.03,0.02,0.02,0.01,0.01,0.01,0.01
COGS and OpEx per Square Foot,489.44,504.13,516.73,529.65,540.24,551.05,559.31,567.7,573.38,579.11
COGS and OpEx per Square Foot Growth Rate,0.03,0.03,0.03,0.03,0.02,0.02,0.01,0.01,0.01,0.01
Maintenance CapEx per Square Foot,9.36,9.64,9.89,10.13,10.34,10.54,10.7,10.86,10.97,11.08
Maintenance CapEx per Square Foot Growth Rate,0.03,0.03,0.03,0.03,0.02,0.02,0.01,0.01,0.01,0.01
D&A per Square Foot,10.2,10.45,10.66,10.87,11.04,11.2,11.31,11.43,11.52,11.61


In [16]:
el.concat([historical_ufcf_df, projected_ufcf_df]).evaluate().transpose(
    include_header=True, header_name="Name", column_names=years
)

Name,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,column_10,column_11,column_12
years,2019.0,2020.0,2021.0,2022.0,2023.0,2024.0,2025.0,2026.0,2027.0,2028.0,2029.0,2030.0,2031.0
Retail Square Feet,1129.0,1129.0,1121.0,1143.42,1166.29,1183.78,1201.54,1213.55,1225.69,1231.82,1237.98,1244.17,1250.39
Retail Square Feet Growth Rate,None,0.0,-0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
Sales per Square Foot,452.02,460.52,495.3,510.16,525.47,538.6,552.07,563.11,574.37,582.99,591.73,597.65,603.62
Sales per Square Foot Growth Rate,None,0.02,0.08,0.03,0.03,0.03,0.03,0.02,0.02,0.01,0.01,0.01,0.01
COGS and OpEx per Square Foot,432.57,442.3,475.19,489.44,504.13,516.73,529.65,540.24,551.05,559.31,567.7,573.38,579.11
COGS and OpEx per Square Foot Growth Rate,None,0.02,0.07,0.03,0.03,0.03,0.03,0.02,0.02,0.01,0.01,0.01,0.01
Maintenance CapEx per Square Foot,8.93,9.48,9.09,9.36,9.64,9.89,10.13,10.34,10.54,10.7,10.86,10.97,11.08
Maintenance CapEx per Square Foot Growth Rate,None,0.06,-0.04,0.03,0.03,0.03,0.03,0.02,0.02,0.01,0.01,0.01,0.01
D&A per Square Foot,9.46,9.73,9.95,10.2,10.45,10.66,10.87,11.04,11.2,11.31,11.43,11.52,11.61


# TODO

1. Displaying the formula-based ExcelFrame is horribly broken - it assumes that the columns the cells refer to always exist in the table, and it's broken with transpose, too. Fix this.
2. ~~df.select()~~
3. df.transpose() is still a bit ugly - Is there a way to pass column names based on one of my existing columns more easily?
4. `with_columns()` doesn't do any coping on self yet it's a modifying API - think about this.
   1. Ideally, you'd want some sort of a shallow copy.